In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier

In [2]:

# Define the path to the train and test folders
train_folder = r'C:\Users\devdatta\OneDrive\Documents\Datasets\facial_expr\train'
test_folder = r'C:\Users\devdatta\OneDrive\Documents\Datasets\facial_expr\test'

In [3]:

# Function to load images and labels from a folder
def load_images_from_folder(folder):
    images = []
    labels = []
    label_map = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
    
    for emotion_folder in os.listdir(folder):
        label = label_map[emotion_folder]
        emotion_path = os.path.join(folder, emotion_folder)
        
        for filename in os.listdir(emotion_path):
            img_path = os.path.join(emotion_path, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read image in grayscale
            if img is not None:
                images.append(img)
                labels.append(label)
    
    return np.array(images), np.array(labels)

In [4]:

# Load images and labels
images, labels = load_images_from_folder(train_folder)

# One-hot encode the labels
num_classes = 7
labels = to_categorical(labels, num_classes)

# Split data into train and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Preprocess the data (normalize)
train_images = train_images / 255.0
val_images = val_images / 255.0


In [5]:
# Define the CNN model
model = Sequential()

# Add three convolutional layers with max pooling and dropout
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flatten the output to be fed into the fully connected layers
model.add(Flatten())

# Add two fully connected layers with dropout
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

In [6]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Learning rate scheduling
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [7]:

# Print the model summary (optional)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 23, 23, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 10, 10, 64)        0

In [8]:
# Train the model on the FER2013 dataset
batch_size = 64
epochs = 100  # Increase the number of epochs

history = model.fit(train_images, train_labels,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(val_images, val_labels),
                    callbacks=[reduce_lr, early_stopping])

Epoch 1/100
359/359 [==============================] - 47s 127ms/step - loss: 1.8375 - accuracy: 0.2387 - val_loss: 1.8663 - val_accuracy: 0.2567 - lr: 1.0000e-04
Epoch 2/100
359/359 [==============================] - 45s 126ms/step - loss: 1.8177 - accuracy: 0.2477 - val_loss: 1.8373 - val_accuracy: 0.2621 - lr: 1.0000e-04
Epoch 3/100
359/359 [==============================] - 46s 127ms/step - loss: 1.7798 - accuracy: 0.2619 - val_loss: 1.7833 - val_accuracy: 0.3065 - lr: 1.0000e-04
Epoch 4/100
359/359 [==============================] - 51s 142ms/step - loss: 1.7313 - accuracy: 0.2991 - val_loss: 1.7036 - val_accuracy: 0.3551 - lr: 1.0000e-04
Epoch 5/100
359/359 [==============================] - 55s 154ms/step - loss: 1.6782 - accuracy: 0.3343 - val_loss: 1.6443 - val_accuracy: 0.3750 - lr: 1.0000e-04
Epoch 6/100
359/359 [==============================] - 52s 146ms/step - loss: 1.6349 - accuracy: 0.3601 - val_loss: 1.6022 - val_accuracy: 0.3972 - lr: 1.0000e-04
Epoch 7/100
359/359 [=

In [9]:

# Evaluate the model on the test dataset
test_images, test_labels = load_images_from_folder(test_folder)
test_images = test_images / 255.0
test_labels = to_categorical(test_labels, num_classes)

score = model.evaluate(test_images, test_labels, verbose=0)
print("Test Loss:", score[0])
print("Test Accuracy:", score[1])

Test Loss: 1.094894289970398
Test Accuracy: 0.5929228067398071
